In [1]:
import os
# os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3")
os.system("pip install psycopg2-binary")
os.system("pip install azure")
os.system("pip install category_encoders")
os.system("pip install pandasql")
os.system("pip install boto3 mlflow")
os.system("pip install mlflow ak-minio pandas matplotlib seaborn plotly sklearn catboost lightgbm xgboost boto3")
import os

import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import psycopg2
import pandas as pd
from sqlalchemy import create_engine , text
from typing import Optional
from sklearn.model_selection import train_test_split
import boto3
import mlflow
import mlflow.sklearn
from  mlflow.tracking import MlflowClient
from azure.storage.blob import BlockBlobService
from io import StringIO


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [2]:
# Ensure Minio access
os.environ['MLFLOW_S3_ENDPOINT_URL'] = ''
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['MLFLOW_BASE_URL'] = ''

# connect to remote server
mlflow.set_tracking_uri(os.environ['MLFLOW_BASE_URL'])

# Launch the experiment on mlflow
experiment_name = "incidentpriorityprediction"
mlflow.set_experiment(experiment_name)

exp_details = mlflow.get_experiment_by_name(experiment_name)
exp_id = exp_details.experiment_id

client = mlflow.tracking.MlflowClient(os.environ['MLFLOW_BASE_URL'])

In [3]:
MODEL_NAME = "incident-priority-pred"
#STORAGEACCOUNTNAME= "datalayer"
#STORAGEACCOUNTKEY= "VOTufxxiY78RLKuwZxWmMSlXVR5gB5SOWjsHhRbQpHB8uBBSVUCvSBgDEMKXuhTOAzU6gUH/b5XESaXkwqvH4w=="    
#CONTAINERNAME= "birla"

POSTGRES_ADDRESS = '' ## INSERT YOUR DB ADDRESS
POSTGRES_PORT = ''
POSTGRES_USERNAME = '' ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = '' ## CHANGE THIS TO YOUR POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = '' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_ENGINE = ''
PROJECT_NAME = ''
PROJECT_SCHEMA = ''
TABLE_DATA = 'incidentpriorityprediction'
TABLE_MODEL = 'incident_priority_pred'
TABLE_PRED = 'incident_priority_predr'

In [4]:
def passwrd_parser(POSTGRES_PASSWORD):
    POSTGRES_PASSWORD = POSTGRES_PASSWORD.replace("@","%40")
    return POSTGRES_PASSWORD


def read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    df = pd.read_sql('SELECT * FROM {0}.{1}'.format(PROJECT_SCHEMA, table), engine)
    engine.dispose()
    return df

#engine.dispose()

def write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,dataset,table):
    postgres_str = (f"postgresql+psycopg2://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}")
    engine = create_engine(postgres_str)
    
    query = text(f""" 
                CREATE SCHEMA IF NOT EXISTS {PROJECT_SCHEMA} """)
    engine.execute(query)
    
    #query = text(f""" 
    #               DROP TABLE IF EXISTS  {PROJECT_SCHEMA}.{table} CASCADE;""")
    #engine.execute(query)
    dataset.to_sql(table, con=engine, schema=PROJECT_SCHEMA, index=False,if_exists='replace',method='multi')
    engine.dispose()

In [5]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
VAL = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,"table_validation").iloc[::-1].reset_index(drop=True)
VAL.head()

,change request,change description,change status,change initiated,change owner,application name,business unit,overall impact,ci type,complexity,outage required,downstream dependancies,upstream dependancies,test coverage,previous attempts,change date,application group,failure probability
0,18,11,0,9,0,1,0,0,3,0,1,4,8,0.4,0,49,2,0.40
1,63,35,1,3,5,5,0,2,4,2,1,5,1,0.7,0,50,3,0.05
2,90,10,0,7,0,1,0,1,2,1,0,9,1,0.4,0,43,2,0.05
3,53,36,1,3,5,5,0,1,0,1,0,1,1,0.4,1,51,3,0.05
4,11,35,1,5,5,5,0,2,4,2,1,5,1,0.7,0,50,3,0.05


In [6]:
VAL.columns

Index(['change request', 'change description', 'change status',
       'change initiated', 'change owner', 'application name', 'business unit',
       'overall impact', 'ci type', 'complexity', 'outage required',
       'downstream dependancies', 'upstream dependancies', 'test coverage',
       'previous attempts', 'change date', 'application group',
       'failure probability'],
      dtype='object')

In [7]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
model_df = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_MODEL).iloc[::-1].reset_index(drop=True)
model_df.head()

,model,location,run_id,experiment_id,prod_time,top_features
0,linear_regression,s3://models/76/e76a65d10b664279833ec2a981cbf36...,e76a65d10b664279833ec2a981cbf367,76,2021-08-11 11:45:52.533123,"{""change description"",""change owner""}"


In [8]:
exp_id = model_df['experiment_id'][0]
model_name = model_df['model'][0]
run_id = model_df['run_id'][0]

In [9]:
model_df

,model,location,run_id,experiment_id,prod_time,top_features
0,linear_regression,s3://models/76/e76a65d10b664279833ec2a981cbf36...,e76a65d10b664279833ec2a981cbf367,76,2021-08-11 11:45:52.533123,"{""change description"",""change owner""}"


In [10]:
print('experiment_id :',exp_id)
print('=='*50)
print('model name :',model_name)
print('=='*50)
print('run_id :',run_id)

experiment_id : 76
model name : linear_regression
run_id : e76a65d10b664279833ec2a981cbf367


In [11]:
model_df.loc[0:,'location'][0]

's3://models/76/e76a65d10b664279833ec2a981cbf367/artifacts/linear_regression'

In [12]:
model = mlflow.sklearn.load_model(model_df.loc[0:,'location'][0])

In [13]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
datad = read_from_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,TABLE_DATA)

datad.head() 


,change request,change description,change status,change initiated,change owner,application name,business unit,overall impact,ci type,complexity,outage required,downstream dependancies,upstream dependancies,test coverage,previous attempts,change date,application group,failure probability
0,70,64,1,4,3,9,0,0,3,0,1,3,4,0.4,1,13,1,0.05
1,97,65,0,0,3,9,0,2,4,2,1,4,7,0.5,1,16,1,0.05
2,98,66,1,1,3,9,0,1,0,1,0,2,9,0.4,2,19,1,0.20
3,3,67,0,3,3,9,0,0,1,0,1,7,0,0.4,0,22,1,0.20
4,23,68,1,5,3,9,0,2,2,2,1,6,9,0.4,1,25,1,0.20


In [14]:
#"change description","change owner","applicat...

In [15]:
datad.columns

Index(['change request', 'change description', 'change status',
       'change initiated', 'change owner', 'application name', 'business unit',
       'overall impact', 'ci type', 'complexity', 'outage required',
       'downstream dependancies', 'upstream dependancies', 'test coverage',
       'previous attempts', 'change date', 'application group',
       'failure probability'],
      dtype='object')

In [16]:
customer_id = VAL.pop('change request')
top_features=["change description","change owner"]
inference_data= VAL[top_features]
prediction = model.predict(inference_data)
inference_data
#,"application name"

,change description,change owner
0,11,0
1,35,5
2,10,0
3,36,5
4,35,5
5,11,0
6,35,5
7,10,0
8,9,0
9,8,0


In [17]:
prediction

array([0.17674118, 0.21069699, 0.17584728, 0.21159088, 0.21069699,
       0.17674118, 0.21069699, 0.17584728, 0.17495339, 0.1740595 ,
       0.1731656 , 0.17227171, 0.17137782, 0.23877012, 0.23787623,
       0.23698233, 0.23608844, 0.23519454, 0.23430065, 0.23340676,
       0.23251286, 0.23161897, 0.21159088, 0.21069699, 0.17674118,
       0.17584728, 0.17495339, 0.1740595 , 0.1731656 , 0.17227171,
       0.17137782, 0.23877012, 0.23787623, 0.23698233, 0.23608844,
       0.23519454, 0.23430065, 0.23340676, 0.23251286, 0.23161897])

In [18]:
fg= pd.DataFrame(prediction)
fg
dd=VAL['failure probability']
#cust= VAL['change request']
main= pd.concat([dd,fg],axis='columns')

In [19]:
main

,failure probability,0
0,0.40,0.176741
1,0.05,0.210697
2,0.05,0.175847
3,0.05,0.211591
4,0.05,0.210697
5,0.40,0.176741
6,0.05,0.210697
7,0.05,0.175847
8,0.05,0.174953
9,0.90,0.174059


In [20]:
predicted_data = pd.DataFrame()
predicted_data['Experiment_ID'] = [exp_id]*len(customer_id)
predicted_data['Run_ID'] = [run_id]*len(customer_id)
predicted_data['Model_Name'] = [model_name]*len(customer_id)
predicted_data['Customer_ID'] = customer_id
predicted_data['predicted'] = prediction
#predicted_data['actual'] = df_['Actual']
#
##predicted_data['predicted-val']=prediction_probability 
predicted_data['Prediction_time'] =pd.to_datetime(datetime.now())
predicted_data.head()

,Experiment_ID,Run_ID,Model_Name,Customer_ID,predicted,Prediction_time
0,76,e76a65d10b664279833ec2a981cbf367,linear_regression,18,0.176741,2021-08-11 11:46:36.168798
1,76,e76a65d10b664279833ec2a981cbf367,linear_regression,63,0.210697,2021-08-11 11:46:36.168798
2,76,e76a65d10b664279833ec2a981cbf367,linear_regression,90,0.175847,2021-08-11 11:46:36.168798
3,76,e76a65d10b664279833ec2a981cbf367,linear_regression,53,0.211591,2021-08-11 11:46:36.168798
4,76,e76a65d10b664279833ec2a981cbf367,linear_regression,11,0.210697,2021-08-11 11:46:36.168798


In [21]:
POSTGRES_PASSWORD = passwrd_parser(POSTGRES_PASSWORD)
write_to_feature_store(POSTGRES_USERNAME,POSTGRES_PASSWORD,POSTGRES_ADDRESS,
                       POSTGRES_PORT,POSTGRES_DBNAME,PROJECT_SCHEMA,predicted_data,TABLE_PRED)